In [4]:
import json
from collections import Counter

# Define the function to count answer types
def count_answer_types(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)

    # Count each answer type
    answer_types = [item['answer_type'] for item in data if 'answer_type' in item]
    return Counter(answer_types), len(data)

# Paths to the VizWiz annotation files
files = {
    'train': '/Users/zagaraa/Documents/GitHub/visionaid-vqa/data/balanced_subset/annotations/train.json',
    'val': '/Users/zagaraa/Documents/GitHub/visionaid-vqa/data/balanced_subset/annotations/val.json',
    'test': '/Users/zagaraa/Documents/GitHub/visionaid-vqa/data/balanced_subset/annotations/test.json',
}

# Aggregate and display counts
for split, path in files.items():
    counts, total = count_answer_types(path)
    print(f"{split.capitalize()} set counts (Total: {total}):")
    for k in ['yes/no', 'number', 'other', 'unanswerable']:
        print(f"  {k}: {counts.get(k, 0)}")
    print()

Train set counts (Total: 1999):
  yes/no: 96
  number: 33
  other: 1178
  unanswerable: 692

Val set counts (Total: 283):
  yes/no: 13
  number: 4
  other: 168
  unanswerable: 98

Test set counts (Total: 569):
  yes/no: 27
  number: 9
  other: 336
  unanswerable: 197

